In [ ]:
import os
import json
import torch
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from torch.utils.data import DataLoader
from sklearn.metrics import mean_absolute_percentage_error

from src.util import *
from src.arima import ARIMAModel
from src.sarima import SARIMAModel
from src.dense import DenseClassifier, DenseNN
from src.lstm import LSTM
from src.cnn import CNN1D, CNNLSTM

In [ ]:
ROOT = os.getcwd()
MODELS = os.path.join(ROOT, 'models')
METRICS = os.path.join(ROOT, 'metrics')
DATA = os.path.join(ROOT, 'dataset')
FIGURES = os.path.join(ROOT, 'figures')

In [ ]:
# Detectar si hay GPU disponible y configurar el dispositivo
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"Using device: {device}")

## Construir Dataset

In [ ]:
# Read the CSV file from the dataset folder
file_path = os.path.join(DATA, 'TrafficTwoMonth.csv')
df = pd.read_csv(file_path)

# Create datetime column
df = create_datetime_column(df)
df.drop(['Time', 'Date', 'Day of the week'], axis=1, inplace=True)

# Clasificación del Estado del Tráfico

En esta sección abordaremos un problema de clasificación multiclase para predecir el estado del tráfico. 

## Clases a Predecir
El estado del tráfico se clasifica en 4 categorías:
- **Low**: Tráfico bajo
- **Normal**: Tráfico normal 
- **High**: Tráfico alto
- **Heavy**: Tráfico muy alto/congestionado

## Datos de Entrada
Como variables de entrada se utilizan conteos de diferentes tipos de vehículos, medidos cada 15 minutos:
- Cantidad de automóviles
- Cantidad de motocicletas 
- Cantidad de colectivos/buses
- Cantidad de camiones

## Modelo y División de Datos
Para resolver este problema de clasificación se implementará una red neuronal. Los datos se dividirán de la siguiente manera:
- 70% para entrenamiento (train)
- 15% para validación (validation)
- 15% para prueba (test)

Esta división nos permitirá entrenar el modelo, ajustar hiperparámetros y evaluar su rendimiento de manera adecuada en datos no vistos durante el entrenamiento.

In [ ]:
# Preparar los datos de entrada (X)
input_columns = ['CarCount', 'BikeCount', 'BusCount', 'TruckCount']
X = df[input_columns].values

# Escalar los datos de entrada usando DataScaler por columna
scalers = []
X_scaled_list = []

for col in range(X.shape[1]):
    scaler = DataScaler(feature_range=(0, 1))
    X_scaled_col = scaler.fit_transform(X[:, col])
    scalers.append(scaler)
    X_scaled_list.append(X_scaled_col)

# Combinar las columnas escaladas
X_scaled = np.column_stack(X_scaled_list)

# Preparar las etiquetas (y)
class_mapping = {'low': 0, 'normal': 1, 'high': 2, 'heavy': 3}
y = df['Traffic Situation'].map(class_mapping).values

# Convertir a tensores de PyTorch
X_tensor = torch.FloatTensor(X_scaled)
y_tensor = torch.LongTensor(y)

# Mezclar los datos antes de dividir
indices = torch.randperm(len(X_tensor))
X_tensor = X_tensor[indices]
y_tensor = y_tensor[indices]

# Dividir en conjuntos de entrenamiento, validación y test
total_samples = len(X_tensor)
train_size = int(0.7 * total_samples)
val_size = int(0.15 * total_samples)
test_size = total_samples - train_size - val_size

X_train = X_tensor[:train_size]
y_train = y_tensor[:train_size]

X_val = X_tensor[train_size:train_size+val_size]
y_val = y_tensor[train_size:train_size+val_size]

X_test = X_tensor[train_size+val_size:]
y_test = y_tensor[train_size+val_size:]

# Crear DataLoaders
batch_size = 32
train_loader = DataLoader(list(zip(X_train, y_train)), batch_size=batch_size, shuffle=True)
val_loader = DataLoader(list(zip(X_val, y_val)), batch_size=batch_size)
test_loader = DataLoader(list(zip(X_test, y_test)), batch_size=batch_size)

# Calcular los pesos de las clases para manejar el desbalance
class_counts = torch.bincount(y_tensor)
num_classes = len(class_counts)
class_weights = class_counts.sum() / (num_classes * class_counts.float())
class_weights = class_weights.to(device)

print("\nPesos por clase:")
for i, weight in enumerate(class_weights):
    print(f"Clase {i}: {weight:.4f}")
print("--------------------------------")
print(f"Dimensiones de los datos:")
print(f"X_train: {X_train.shape}")
print(f"X_val: {X_val.shape}")
print(f"X_test: {X_test.shape}")
print("--------------------------------")
print(f"Distribución de clases en el conjunto de entrenamiento:")
print(pd.Series(y_train.numpy()).value_counts().sort_index())

In [ ]:
CLASSIFIER = {'params': "classifier.pth",
              'metrics': "classifier_metrics.json",
              'plot_metrics': "classifier_metrics.png",
              'confusion_matrix': "classifier_confusion_matrix.png"}

# Crear el modelo
input_size = len(input_columns)  # 4 features
hidden_sizes = [32, 32]  # Dos capas ocultas
output_size = 4  # 4 clases
classifier = DenseClassifier(input_size=input_size, 
                             hidden_sizes=hidden_sizes, 
                             num_classes=output_size, 
                             device=device,
                             class_weights=class_weights,
                             init_method='xavier_normal')

# Cargar métricas si existen, sino definirlas
metrics_path = os.path.join(METRICS, CLASSIFIER['metrics'])
if os.path.exists(metrics_path):
    with open(metrics_path, 'r') as f:
        traffic_classifier_metrics = json.load(f)
else:
    traffic_classifier_metrics = {'epochs': [], 
                                  'loss': {'train': [], 'val': []}, 
                                  'accuracy': {'train': [], 'val': []}}

In [ ]:
# Definir optimizador
optimizer = torch.optim.RMSprop(classifier.parameters(), lr=1e-4, weight_decay=1e-5)
# Cargar modelo si existe
model_path = os.path.join(MODELS, CLASSIFIER['params'])
if os.path.exists(model_path):
    classifier.load_state_dict(torch.load(model_path))
# Entrenar modelo
traffic_classifier_metrics = classifier.fit(train_loader=train_loader, 
                                            val_loader=val_loader, 
                                            optimizer=optimizer, 
                                            num_epochs=50, 
                                            print_every=10,
                                            metrics=traffic_classifier_metrics)
# Guardar métricas
with open(os.path.join(METRICS, CLASSIFIER['metrics']), 'w') as f:
    json.dump(traffic_classifier_metrics, f)
# Guardar modelo
torch.save(classifier.state_dict(), os.path.join(MODELS, CLASSIFIER['params']))
# Graficar pérdidas
plot_metrics(history=traffic_classifier_metrics, 
            title='Métricas - Traffic Classifier', 
            save_path=os.path.join(FIGURES, CLASSIFIER['plot_metrics']))
# Matriz de confusión
cm, metrics = classifier.compute_confusion_matrix(test_loader, 
                                                 plot=True, 
                                                 save_path=os.path.join(FIGURES, CLASSIFIER['confusion_matrix']))
# Imprimir métricas
print(metrics)